In [113]:
import sqlite3

conn = sqlite3.connect('TestDB.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - IMPORT_Courses
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSES''')
c.execute('''CREATE TABLE IMPORT_COURSES(
    [term] TEXT,
    [crn] TEXT,
    [catalog_id] TEXT,
    [section] TEXT,
    [credits] TEXT,
    [title] TEXT,
    [meetings] TEXT,
    [timecodes] TEXT,
    [primary_instructor] TEXT,
    [cap] INT,
    [act] INT, 
    [rem] INT,
    [instructor_fname] TEXT,
    [instructor_lname] TEXT,
    [semester] TEXT,
    [year] TEXT)'''
         )
          
# Create table - IMPORT_COURSE_MEETINGS
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS''')
c.execute('''CREATE TABLE IMPORT_COURSE_MEETINGS (
    [crn] TEXT,
    [location] TEXT,
    [day] TEXT,
    [start] TEXT,
    [end] TEXT,
    [term] TEXT, 
    [building] TEXT, 
    [room] TEXT)'''
         )
        
# Create table - IMPORT_COURSE_CATALOG
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_CATALOG''')

c.execute('''CREATE TABLE IMPORT_COURSE_CATALOG (
    [program_code] TEXT,
    [program_name] TEXT,
    [catalog_ID] TEXT,
    [course_title] TEXT,
    [credits] TEXT,
    [prereqs] TEXT,
    [coreqs] TEXT,
    [fees] TEXT,
    [attributes] TEXT,
    [description] TEXT,
    [EffectiveTerm] TEXT)'''
         )
                 
conn.commit()

# Note that the syntax to create new tables should only be used once in the code (unless you dropped the table/s at the end of the code). 
# The [generated_id] column is used to set an auto-increment ID for each record
# When creating a new table, you can add both the field names as well as the field formats (e.g., Text)

conn.close()




In [114]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///TestDB.db")
IMPORT_Courses = pd.read_sql_table('IMPORT_COURSES',engine)
display(IMPORT_Courses.head())
IMPORT_COURSE_CATALOG = pd.read_sql_table('IMPORT_COURSE_CATALOG',engine)
display(IMPORT_COURSE_CATALOG.head())
IMPORT_COURSE_MEETINGS = pd.read_sql_table('IMPORT_COURSE_MEETINGS',engine)
display(IMPORT_COURSE_MEETINGS.head())

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year


,program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,EffectiveTerm


,crn,location,day,start,end,term,building,room


In [7]:
%load_ext sql

# initialize a database connection; may have to adjust username and password 

%sql sqlite:///TestDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @TestDB.db'

In [13]:
%%sql
SELECT  *
FROM IMPORT_COURSE_CATALOG

 * sqlite:///TestDB.db
Done.


program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,EffectiveTerm


In [75]:
import glob
import os
import pandas as pd
#creating coureses df
##https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

semester_folders = ['Fall2014',
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']



# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames
for semester in semester_folders:
    # the path to your csv file directory
    mycsvdir = 'SourceData/{}/courses.csv'.format(semester)
# read all the csv files in that directory (assuming they have the extension courses.csv)

    df = pd.read_csv(mycsvdir)
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)

# print out to a new csv file
result.to_csv('courses_all.csv')


In [76]:
result.head(1).T

,0
term,Fall2014
crn,70384
catalog_id,AC 0011
section,C01
credits,3.000
title,Introduction to Financial Accounting
meetings,"[{'days': 'TF', 'times': '0800am-0915am', 'dat..."
timecodes,['TF 0800am-0915am 09/02-12/08 DSB 105']
primary_instructor,Michael P. Coyne
cap,0


In [77]:
import json
import pandas as pd
import re
def parse_column(data):
    try:
        return json.loads(data)
    except Exception as e:
        print(e)
        return None



from pandas.io.json import json_normalize
df = pd.read_csv('courses_all.csv' ,header=0)

df[['instructor_fname','instructor_lname']] = df['primary_instructor'].str.split('  ',expand=True,)
df[['semester','year']] = df['term'].str.split('2',expand=True,)
df['year'] = '2'+df['year']

#df['days'] =df['meetings'][0].strip("\n ' '").split(' ')[1].replace(',',"").replace("'","")
#df['times'] =df['meetings'][0].strip("\n ' '").split(' ')[3].replace(',',"").replace("'","")
#df['dates'] =df['meetings'][0].strip("\n ' '").split(' ')[5].replace(',',"").replace("'","")
#df['start'] =df['meetings'][0].strip("\n ' '").split(' ')[5].split("-")[0].replace(',',"").replace("'","")
#df['end'] =df['meetings'][0].strip("\n ' '").split(' ')[5].split("-")[1].replace(',',"").replace("'","")

#df['location_building'] =df['meetings'][0].strip("\n ' '").split(' ')[7].replace(',',"").replace("'","")
#df['location_building_room'] =df['meetings'][0].strip("\n ' '").split(' ')[8].replace("}]","").replace("'","")
#pd.concat( [dw['meetings'].apply(pd.Series)], axis=1)
#df['meetings'][0]
#pd.concat([df.drop('meetings', axis=1), pd.DataFrame(df['meetings'].tolist())], axis=1)
df.head(2).T



,0,1
Unnamed: 0,0,1
term,Fall2014,Fall2014
crn,70384,70385
catalog_id,AC 0011,AC 0011
section,C01,C02
credits,3.000,3.000
title,Introduction to Financial Accounting,Introduction to Financial Accounting
meetings,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...","[{'days': 'TF', 'times': '0930am-1045am', 'dat..."
timecodes,['TF 0800am-0915am 09/02-12/08 DSB 105'],['TF 0930am-1045am 09/02-12/08 DSB 105']
primary_instructor,Michael P. Coyne,Michael P. Coyne


In [112]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,15937.0,7968.000000,4600.759955,0.0,3984.0,7968.0,11952.0,15936.0
crn,15937.0,55172.299178,19308.879968,10893.0,35426.0,57145.0,74541.0,79215.0
cap,15937.0,17.992784,12.671906,0.0,10.0,19.0,25.0,500.0
act,15937.0,15.578591,10.107690,-1.0,7.0,15.0,23.0,90.0
rem,15937.0,2.477317,12.709781,-75.0,0.0,1.0,5.0,999.0


In [108]:
for i in df.columns[1:]:
    print("       ", df[i].name)
    display(df[i].value_counts().to_frame('count'))
    print('--------')

        term


,count
Fall2014,1522
Fall2018,1519
Fall2016,1491
Fall2017,1479
Fall2015,1475
Spring2019,1473
Spring2017,1435
Spring2015,1421
Spring2018,1405
Spring2016,1387


--------
        crn


,count
32768,2
34156,2
75160,2
31890,2
75918,2
...,...
77756,1
34745,1
57266,1
77740,1


--------
        catalog_id


...

--------
        semester


,count
Fall,7486
Spring,7121
Summer,1224
Winter,105
SpringBreak,1


--------
        year


,count
2018,3265
2015,3237
2017,3224
2016,3216
2014,1522
2019,1473


--------


# imput meetings and timecodes columns
 ---

In [80]:
df.meetings.loc[df.meetings == '[]'].count()

2491

In [81]:
import numpy as np
df.meetings.loc[df.meetings == '[]'] = np.nan

In [101]:
df.meetings.isna().sum()

2491

In [48]:

df.timecodes.loc[df.timecodes == '[]'].count()

2491

In [49]:
df.timecodes.loc[df.timecodes == '[]']= np.nan

In [50]:
df.timecodes.isna().sum()

2491